# AiiDAlab Quantum ESPRESSO Plugin list
This [repository](https://github.com/aiidalab/aiidalab-qe-plugin-registry) contains the source of the official registry of AiiDAlab Quantum ESPRESSO plugins. If you are starting to develop a new plugin or if you already have one, please register it there.


In [ ]:
import requests
import yaml

# URL of the YAML file
filepath = 'https://raw.githubusercontent.com/aiidalab/aiidalab-qe-plugin-registry/main/plugins.yaml'

# Fetch the contents of the URL
response = requests.get(filepath)

# Check if the request was successful
if response.status_code == 200:
    # Load the YAML content
    data = yaml.safe_load(response.content)
    # Now 'data' contains the YAML file's contents as a Python object
else:
    print(f"Failed to fetch the YAML file: HTTP {response.status_code}")

In [ ]:
import subprocess
from threading import Thread

import ipywidgets as ipw
from IPython.display import display


def is_package_installed(package_name):
    import importlib
    package_name = package_name.replace('-', '_')
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False


def stream_output(process, output_widget):
    """Reads output from the process and forwards it to the output widget."""
    while True:
        output = process.stdout.readline()
        if process.poll() is not None and output == '':
            break
        if output:
            output_widget.value += f"""<div style="background-color: #3B3B3B; color: #FFFFFF;">{output}</div>"""


def execute_command_with_output(command, output_widget, install_btn, remove_btn, action="install"):
    """Execute a command and stream its output to the given output widget."""
    output_widget.value = ""  # Clear the widget
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    # Create a thread to read the output stream and write it to the output widget
    thread = Thread(target=stream_output, args=(process, output_widget))
    thread.start()
    thread.join()  # Wait for the thread to finish

    if process.returncode == 0 and action == "install":
        output_widget.value += """<div style="background-color: #3B3B3B; color: #008000;">Command executed successfully.</div>"""
        install_btn.disabled = True
        remove_btn.disabled = False
        return True
    elif process.returncode == 0 and action == "remove":
        output_widget.value += """<div style="background-color: #3B3B3B; color: #008000;">Command executed successfully.</div>"""
        install_btn.disabled = False
        remove_btn.disabled = True
        return True
    else:
        output_widget.value += """<div style="background-color: #3B3B3B; color: #FF0000;">Command failed.</div>"""
        return False


def install_package(pip, github, output_container, install_btn, remove_btn, accordion, index):
    if pip:
        command = ["pip", "install", pip]
    else:
        command = ["pip", "install", "git+" + github]
    result = execute_command_with_output(command, output_container, install_btn, remove_btn)
    if result:
        # restart daemon
        accordion.set_title(index, f"{accordion.get_title(index)[:-2]} ✅")
        command = ["verdi", "daemon", "restart"]
        subprocess.run(command, capture_output=True, shell=False)


def remove_package(package_name, output_container, install_btn, remove_btn, accordion, index):
    package_name = package_name.replace('-', '_')
    command = ["pip", "uninstall", "-y", package_name]
    result = execute_command_with_output(command, output_container, install_btn, remove_btn, action="remove")
    if result:
        accordion.set_title(index, f"{accordion.get_title(index)[:-2]} ☐")
        command = ["verdi", "daemon", "restart"]
        subprocess.run(command, capture_output=True, shell=False)


accordion = ipw.Accordion()

for i, (plugin_name, plugin_data) in enumerate(data.items()):
    installed = is_package_installed(plugin_name)
    
    # Output container with customized styling
    output_container = ipw.HTML(
        value="""
        <div style="background-color: #3B3B3B; color: #FFFFFF; height: 100%; overflow: auto;">
        </div>
        """,
        layout=ipw.Layout(
            max_height='250px', 
            overflow='auto',
            border='2px solid #CCCCCC'
        )
    )
    
    details = f"Author: {plugin_data.get('author', 'N/A')}<br>" \
              f"Description: {plugin_data.get('description', 'No description available')}<br>"
    if 'documentation' in plugin_data:
        details += f"Documentation: <a href='{plugin_data['documentation']}' target='_blank'>Visit</a><br>"
    if 'github' in plugin_data:
        details += f"Github: <a href='{plugin_data.get('github')}' target='_blank'>Visit</a>"

    install_btn = ipw.Button(description="Install", button_style='success', disabled=installed)
    remove_btn = ipw.Button(description="Remove", button_style='danger', disabled=not installed)

    install_btn.on_click(lambda btn, pip=plugin_data.get('pip', None), github=plugin_data.get('github', ''), oc=output_container, ib=install_btn, rb=remove_btn, ac=accordion, index=i: install_package(pip, github, oc, ib, rb, ac, index))
    remove_btn.on_click(lambda btn, pn=plugin_name, oc=output_container, ib=install_btn, rb=remove_btn, ac=accordion, index=i: remove_package(pn, oc, ib, rb, ac, index))

    box = ipw.VBox([
        ipw.HTML(details),
        ipw.HBox([install_btn, remove_btn]),
        output_container  # Include the output container in the VBox
    ])

    title_with_icon = f"{plugin_name} {'✅' if installed else '☐'}"
    accordion.set_title(i, title_with_icon)
    accordion.children = list(accordion.children) + [box]

display(accordion)